In [4]:
%run GpuOptions.ipynb
%run BuildTrainingData.ipynb
%run TrendTools.ipynb

Found 2929 images belonging to 4 classes.
Found 731 images belonging to 4 classes.


In [7]:
import os
import tensorflow as tf
import common.model as m

classes = training_data_generator.class_indices
data, _ = training_data_generator.next()
shape = data[0].shape
inputs = tf.keras.Input(shape=shape)
outputs = m.chained(
    tf.keras.layers.add([
        m.chained(
            inputs,
            tf.keras.layers.Lambda(
                lambda image: tf.image.rgb_to_grayscale(image)),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(32),
        ),
        m.chained(
            inputs,
            tf.keras.layers.Lambda(
                lambda image: tf.image.rgb_to_grayscale(image)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, activation='relu'),
            tf.keras.layers.MaxPooling2D(2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(32),
        ),
    ]),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(classes)),
    tf.keras.layers.Activation('softmax'))
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.build(input_shape=(None, *shape))
model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.01),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'])
if os.path.exists('isthemountainout.h5'):
    model.load_weights('isthemountainout.h5')
elif os.path.exists('isthemountainout.best.h5'):
    model.load_weights('isthemountainout.best.h5')
model.summary()

Model: &quot;functional_5&quot;
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 224, 224, 1)  0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 224, 224, 1)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 222, 222, 64) 640         lambda_4[0][0]                   
_____________________________________________________________________________

In [8]:
import time
!rm -rf logs
time.sleep(2)

model.fit(
    training_data_generator,
    epochs=700,
    verbose=True,
    steps_per_epoch=training_data_generator.samples // 
        training_data_generator.batch_size,
    validation_data=validation_data_generator,
    validation_steps=validation_data_generator.samples // 
        validation_data_generator.batch_size,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            'isthemountainout.best.h5',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=True),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=50,
            restore_best_weights=True,
            verbose=True),
        tf.keras.callbacks.CSVLogger(os.path.join('logs', 'isthemountainout.training.csv')),
        tf.keras.callbacks.TensorBoard(
            log_dir='logs', update_freq='batch', write_images=True, write_graph=True, embeddings_freq=10),
        m.LogConfusionMatrixCallback(
            model=model,
            datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)\
                .flow_from_directory(data_directory, batch_size=3096, shuffle=True, target_size=image_size),
            logdir=os.path.join('logs', 'image'))
    ])

Found 3660 images belonging to 4 classes.
Epoch 1/700
91/91 [==============================] - ETA: 0s - loss: 0.9229 - accuracy: 0.8447
Epoch 00001: val_accuracy improved from -inf to 0.78409, saving model to isthemountainout.best.h5
91/91 [==============================] - 51s 560ms/step - loss: 0.9229 - accuracy: 0.8447 - val_loss: 0.9846 - val_accuracy: 0.7841
Epoch 2/700
91/91 [==============================] - ETA: 0s - loss: 0.8896 - accuracy: 0.8840
Epoch 00002: val_accuracy did not improve from 0.78409
91/91 [==============================] - 43s 468ms/step - loss: 0.8896 - accuracy: 0.8840 - val_loss: 1.0284 - val_accuracy: 0.7401
Epoch 3/700
91/91 [==============================] - ETA: 0s - loss: 0.8869 - accuracy: 0.8878
Epoch 00003: val_accuracy did not improve from 0.78409
91/91 [==============================] - 49s 535ms/step - loss: 0.8869 - accuracy: 0.8878 - val_loss: 1.0004 - val_accuracy: 0.7628
Epoch 4/700
91/91 [==============================] - ETA: 0s - loss: 

&lt;tensorflow.python.keras.callbacks.History at 0x1f08b132400&gt;